In [1]:
!pip install langchain langchain-community zipfile


ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for zipfile


In [4]:
import zipfile
import os

# Path to the zip file in your Downloads folder on C drive
zip_path = "C:/Users/Mr.Laptop point/Documents/mimic-iv-ext-direct-1.0.0.zip"

# Destination directory locally
extract_path = "./data"  # This will create/extract to a 'data' folder in your current working directory

# Create the destination directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [5]:
# Install required packages (run this cell in Jupyter with !)
!pip install langchain-community
!pip install langchain



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install jq


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.document_loaders import DirectoryLoader, JSONLoader

def extract_text(content):
    """Recursively extract and flatten all text content into a single string."""
    if isinstance(content, str):
        return content
    elif isinstance(content, dict):
        return " ".join(f"{k}: {extract_text(v)}" for k, v in content.items())
    elif isinstance(content, list):
        return " ".join(extract_text(i) for i in content)
    return ""

# Define the parent directory (local path after extraction)
parent_directory = extract_path  # Use the local extract_path from the previous cell

# Load JSON files recursively
loader = DirectoryLoader(
    parent_directory,
    glob="**/*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": "{text: .}", "text_content": False}  # Load entire JSON as a dictionary
)

# Load documents
documents = loader.load()

# Apply text extraction
for doc in documents:
    doc.page_content = extract_text(doc.page_content)  # Convert dictionary to string

print(f"Loaded {len(documents)} documents successfully!")
if documents:  # Check if documents list is not empty
    print(documents[0].page_content)  # Print sample content (first document)

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize RecursiveCharacterTextSplitter with appropriate chunk size and overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)

# Function to split document into structured chunks based on headings
def split_document_into_chunks(doc_content):
    # Split the document at headings/subheadings
    chunks = splitter.split_text(doc_content)
    return chunks

# Split all documents into chunks
all_chunks = []
if documents:  # Ensure there are documents to process
    for doc in documents:
        chunks = split_document_into_chunks(doc.page_content)
        all_chunks.extend(chunks)

    # Print results
    print(f"Number of chunks: {len(all_chunks)}")
    print("First chunk:")
    if all_chunks:  # Check if there are any chunks
        print(all_chunks[0])  # Print first chunk for inspection
    else:
        print("No chunks were created.")
else:
    print("No documents were loaded.")

# Save all_chunks for use in the next cell
chunks_for_faiss = all_chunks  # Rename for clarity and reuse

Loaded 511 documents successfully!
{"text": {"Alzheimer$Intermedia_3": {"Amyloid deposition detected in frontal and temporal regions, one of the key pathological features of Alzheimer's disease and associated with cognitive deterioration$Cause_1": {"Amyloid deposition was detected in areas of the frontal and temporal lobes of the brain.$Input6": {}}, "The hippocampus and frontal lobes were significantly atrophied. Atrophy in these areas is typical of Alzheimer's disease and is closely associated with loss of memory and cognitive function.$Cause_1": {"Significant atrophy of hippocampus and frontal lobes$Input6": {}}, "Suspected Alzheimer$Intermedia_2": {"In Alzheimer's disease, this is due to changes in neurotransmitters due to damage and death of nerve cells in the brain.$Cause_1": {"altered mental status$Input1": {}}, "This can manifest as verbal or physical aggression, mood swings, or threats to others. In people with Alzheimer's disease, aggressive behavior may be linked to damage t

In [ ]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document
import numpy as np
import faiss
import shutil
import os

# Use the chunks created in the previous cell
if not chunks_for_faiss:
    raise ValueError("No chunks were created. Check the previous cells.")

# Create documents for FAISS
documents_for_faiss = [Document(page_content=chunk) for chunk in chunks_for_faiss]

# Extract raw text from the documents
text_data = [doc.page_content for doc in documents_for_faiss]

# Load the pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for your documents
embeddings = model.encode(text_data, convert_to_tensor=False)  # 'convert_to_tensor=False' for numpy array

# Initialize the FAISS index (using L2 distance)
embedding_dimension = embeddings.shape[1]  # This gives the dimensionality of the embeddings
faiss_index = faiss.IndexFlatIP(embedding_dimension)

# Add the embeddings to the FAISS index
faiss_index.add(np.array(embeddings, dtype=np.float32))

# Save the model locally
model_save_path = './all-MiniLM-L6-v2'
model.save(model_save_path)

# Create a zip archive of the model directory
shutil.make_archive('all-MiniLM-L6-v2', 'zip', model_save_path)

print("Model and index saved locally!")

# Save the FAISS index to a file
faiss.write_index(faiss_index, 'faiss_index_file.index')

# Save embeddings to a file
np.save('embeddings_file.npy', embeddings)

print("FAISS index and embeddings saved successfully!")

# Later, if you need to load them back
faiss_index_loaded = faiss.read_index('faiss_index_file.index')
embeddings_loaded = np.load('embeddings_file.npy')

print("FAISS index and embeddings loaded successfully!")

In [9]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Assuming the model, FAISS index, and documents are already loaded or saved from previous cells
# If not, load them back

# Load the FAISS index and embeddings if they were saved
faiss_index = faiss.read_index('faiss_index_file.index')
embeddings = np.load('embeddings_file.npy')

# Load the SentenceTransformer model (if not already in memory)
model = SentenceTransformer('all-MiniLM-L6-v2')  # Reinitialize or load from saved path if needed
# If you saved the model earlier, you can load it like this:
# model = SentenceTransformer('./all-MiniLM-L6-v2')

# Assuming 'documents' is the list of Document objects from your previous cell
# If 'documents' is not in memory, you can recreate it or load it
# For now, we'll assume it's available as 'documents_for_faiss' from your earlier code
try:
    documents = documents_for_faiss  # Use the name from your previous cells
except NameError:
    raise ValueError("Documents not found. Ensure 'documents_for_faiss' is defined from previous cells or reload it.")

# Define the query
query = "Describe the stages of Alzheimer's disease?"

# Create query embedding
query_embedding = model.encode([query], convert_to_tensor=False)

# Perform the similarity search (get top 3 most relevant documents)
D, I = faiss_index.search(np.array(query_embedding, dtype=np.float32), k=3)

# Retrieve the top 3 most relevant documents based on the query
top_3_relevant_docs = [documents[i] for i in I[0]]

# Print the top 3 most relevant chunks (documents)
print("Top 3 Most Relevant Documents:")
for doc in top_3_relevant_docs:
    print(doc.page_content)
    print("\n" + "="*50 + "\n")  # Add separator between documents

# Print the most similar document based on the highest similarity score
most_similar_doc_index = I[0][0]  # The most similar document corresponds to the first result
most_similar_doc = documents[most_similar_doc_index]

print("Most Similar Document:")
print(most_similar_doc.page_content)

Top 3 Most Relevant Documents:
{"text": {"Alzheimer$Intermedia_3": {"It shows decreased glucose metabolism in the temporal and parietal regions, which is a typical imaging feature of Alzheimer's disease and is associated with brain atrophy and functional decline. It is an important basis for directly supporting the diagnosis of Alzheimer's disease.$Cause_1": {"FDG-PET: show reduced glucose metabolism in the temporal-parietal lobes.$Input6": {}}, "Suspected Alzheimer$Intermedia_2": {"Agitation manifests as emotional instability, which is a manifestation of Alzheimer's disease$Cause_1": {"Agitation$Input1": {}}, "Loss of activity may be an early sign of cognitive decline in Alzheimer's disease$Cause_1": {"less active in his iadls$Input2": {}}, "Loss of personal hygiene management ability is a common symptom of Alzheimer's disease$Cause_1": {"stopped being able to take care of his personal hygiene$Input2": {}}, "Forgetfulness is one of the most common symptoms of Alzheimer's disease$Cause

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
def generate_answer(context, query, model, tokenizer):
    # Prepare the input for LLM
    input_text = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    # Generate output
    outputs = model.generate(**inputs, max_length=512)

    # Decode the output
    final_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return final_answer

  

In [11]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np

# 1. Load Embedding Model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # or the model you used earlier

# 2. Load LLM (Flan-T5)
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 3. Define function to generate answer
def generate_answer(context, query, model, tokenizer):
    prompt = f"""Context: {context}
Question: {query}

Instructions:
- Carefully consider negations (words like "not", "never", "negative").
- Ignore any technical symbols like '$cause_1' or '$intermedia_3'. Focus only on medical facts.
- Write a clear, informative answer.
- Do not repeat the same phrase multiple times.
- Focus on summarizing relevant facts from the context.
- If question end with ?, then explain the answer.
- If information is missing, say "Not enough information."

Answer:"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=512)
    final_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return final_answer

# 4. Query
query = "Describe the symptoms of Type2 Diabetes disease?"
# query = "Describe the Type1 Diabetes in explain?"
# query = """Does the patient's history and clinical findings suggest Type I Diabetes?
# Consider the negative ICA test, there;s never blood glucose levels, having never family history about it, none of physical exam findings (thinness), never face hypertension, and no insulin treatment."""
query_embedding = embedding_model.encode([query], convert_to_tensor=False)

# 5. Retrieval: Perform similarity search
D, I = faiss_index.search(np.array(query_embedding, dtype=np.float32), k=3)

# 6. Retrieve top 3 most relevant documents
top_3_relevant_docs = [documents[i] for i in I[0]]

# 7. Print top 3 documents (optional)
for doc in top_3_relevant_docs:
    print(doc.page_content)
    print("\n" + "="*50 + "\n")

# 8. Merge retrieved documents into one context
# Merge top documents, but truncate to max 1500 characters
retrieved_context = " ".join([doc.page_content for doc in top_3_relevant_docs])[:1500]

# 9. Generate final answer using LLM
final_output = generate_answer(retrieved_context, query, generator_model, tokenizer)

print("\n" + "="*50)
print("Final Generated Answer:")
print(final_output)
generator_model.save_pretrained("./medical")
tokenizer.save_pretrained("./medical")

{"text": {"Type I Diabetes$Intermedia_4": {"ICA antibodies are positive in most patients with type I diabetes.$Cause_1": {"ICA-(+)$Input6": {}}, "Diabetes$Intermedia_3": {"Abnormal blood gluose is a diagnostic criteria of diabetes.$Cause_1": {"GLUCOSE-237$Input6": {}}, "Suspected Diabetes$Intermedia_2": {"DKA is a basic symptom.$Cause_1": {"+/- DKA$Input2": {}}, "Family history is a big risk factor.$Cause_1": {"Her grandmother had diabetes.$Input4": {}}}}}, "input1": "Nausea/vomiting\n", "input2": "Patient is a yo woman with DM1 and frequent admissions for nausea, vomting, and abdominal pain +/- DKA who presents recurrent nausea, vomiting, and abdominal pain.  This morning, after she took her medications, within a couple of minutes she became nauseous and vomiting.  She has not been able to eat anything all day due to nausea/vomiting.  She has no abdominal pain, diarrhea, constipation.  She has her chronic back pain.  Zofran ODT did not help her. She received NS, dilaudid, zofran, and 

SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "The requested operation cannot be performed on a file with a user-mapped section open." })

In [12]:
# ------------------ Evaluation and Analysis ------------------

# Ground truth (for testing evaluation)
ground_truth = "Common symptoms of Type 2 Diabetes include excessive thirst, frequent urination, fatigue, blurred vision, and slow healing wounds."

# Sample retrieved docs to compute Precision/Recall (in real-world you'd label these manually)
relevant_keywords = ["excessive thirst", "frequent urination", "blood glucose", "fatigue", "blurred vision"]

# --- A. Retrieval Evaluation ---
retrieved_texts = [doc.page_content.lower() for doc in top_3_relevant_docs]

# Flatten retrieved text and calculate matches
retrieved_text = " ".join(retrieved_texts)

true_positives = sum(1 for keyword in relevant_keywords if keyword in retrieved_text)
precision = true_positives / len(top_3_relevant_docs)
recall = true_positives / len(relevant_keywords)

print("🔎 Retrieval Evaluation")
print(f"Precision@3: {precision:.2f}")
print(f"Recall@3:    {recall:.2f}")

# --- B. Generation Evaluation (Manual / Sample Comparison) ---
print("\n📝 Generation Evaluation")

def evaluate_generation(generated_answer, ground_truth):
    print(f"Generated Answer: {generated_answer}")
    print(f"Ground Truth:     {ground_truth}")
    
    # Manually assign scores from 0 to 1
    relevance = input("Rate Relevance (0-1): ")
    coherence = input("Rate Coherence (0-1): ")
    factual_accuracy = input("Rate Factual Accuracy (0-1): ")
    
    return {
        "Relevance": relevance,
        "Coherence": coherence,
        "Factual Accuracy": factual_accuracy
    }

generation_scores = evaluate_generation(final_output, ground_truth)
print("\n📊 Manual Generation Metrics:", generation_scores)


🔎 Retrieval Evaluation
Precision@3: 0.33
Recall@3:    0.20

📝 Generation Evaluation
Generated Answer: "text": "Type I Diabetes": "ICA antibodies are positive in most patients with type I diabetes.$Cause_1": "GLUCOSE-237$Input6": , "Diabetes": "Intermedia_3": "Abnormal blood gluose is a diagnostic criteria of diabetes.$Cause_1": "GLUCOSE-237$Input6": , "Diabetes": "Anausea/vomitingn", "input1": "Nausea/vomitingn", "input2": "Patient is a yo woman with DM1 and frequent admissions for nausea, vomiting, and abdominal pain +/- DKA who presents recurrent nausea, vomiting, and abdominal pain. This morning, after she took her medications, within a couple of minutes she became nauseous and vomiting. She has not been able to eat anything all day due to nausea/vomiting. She has no abdominal pain, diarrhea, constipation. She has her chronic back pain. Zofran ODT did not help her. She received NS, dilaudid, zofran, and reglan.nufeffnIn the ED, initial VS were: 98 104 136/87 18 100%. Labs were notab

Rate Relevance (0-1):  0
Rate Coherence (0-1):  1
Rate Factual Accuracy (0-1):  0



📊 Manual Generation Metrics: {'Relevance': '0', 'Coherence': '1', 'Factual Accuracy': '0'}
